TODO:


- [x] Сделать так чтобы результат выводился в виде нормального полинома, а не sum alpha \* Полинома лежандра
- [ ] Проверить правильность результата, тем более второго задания (график совсем не сходться)

**Задание 1.1.**

Условие ортогональности невязки к базисным векторам запишется в виде:

$\int_a^b {(f(x) - p_n(x)) x^j dx} = 0 \quad \Rightarrow$

$\int_a^b {p_n(x) x^j dx} = \int_a^b {f(x) x^j dx} \quad \Rightarrow$

$\sum_{k=0}^n {\alpha_k \int_a^b {x^{j + k} dx}} = \int_a^b {f(x) x^j dx} \quad \Rightarrow$

$\sum_{k=0}^n {\alpha_k \frac{b^{j+k+1} - a^{j+k+1}}{j+k+1}} = \int_a^b {f(x) x^j dx} \quad \Rightarrow$

In [73]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

def f(x):
    return np.sqrt(x**2) + np.log(x**2)

a, b = 1, 6
n = 5 

A = np.zeros((n+1, n+1))
for i in range(n+1):
    for j in range(n+1):
        A[i, j] = (b**(i+j+1) - a**(i+j+1)) / (i+j+1)

b_vec = np.zeros(n+1)
for j in range(n+1):
    def integrand(x):
        return f(x) * x**j
    b_vec[j], _ = integrate.quad(integrand, a, b)

alpha = np.linalg.solve(A, b_vec)

print("Коэффициенты полинома:")
for i, coeff in enumerate(alpha):
    print(f"α_{i} = {coeff:.6f}")

Коэффициенты полинома:
α_0 = -2.702505
α_1 = 4.786422
α_2 = -1.330940
α_3 = 0.291664
α_4 = -0.033892
α_5 = 0.001596


**Задание 1.2.**

In [22]:
import numpy as np
import math
from scipy import integrate
from scipy.special import legendre
from scipy.special import chebyt
import matplotlib.pyplot as plt

class LegendreApproximation:    
    def __init__(self, f, a, b, n, weight_func_vers=1):

        self.f = f
        self.a = a
        self.b = b
        self.n = n
        self._weight_func_vers = weight_func_vers
        if weight_func_vers == 1:
            self.weight_func = lambda x: 1
        elif weight_func_vers == 2:
            self.weight_func = lambda x: 1 / np.sqrt(1 - x**2)
            
        self.coefficients = None
        self.polynomial = None
        
        self.absolute_error = None
        
        self.build_polynomial()

    def x_to_t(self, x):
        """Преобразование x in [a, b] -> t in [-1, 1]"""
        return (2*x - (self.a + self.b)) / (self.b - self.a)
    
    def t_to_x(self, t):
        """Преобразование t in [-1, 1] -> x in [a, b]"""
        return (self.b - self.a)/2 * t + (self.a + self.b)/2
    
    def f_t(self, t):
        x = self.t_to_x(t)
        return self.f(x)
    
    def weight_t(self, t):
        x = self.t_to_x(t)
        return self.weight_func(x)

    def build_polynomial(self):
        self.coefficients = np.zeros(self.n + 1)
        
        for k in range(self.n + 1):
            if self._weight_func_vers == 1:
                Pk = 2**k * math.factorial(k) *legendre(k)
            elif self._weight_func_vers == 2:
                Pk = chebyt(k)
            
            # Вычисляем скалярное произведение (f, φ_k)
            def integrand_f_phi(t):
                return self.f_t(t) * Pk(t) * self.weight_t(t)
            
            integral_f_phi, _ = integrate.quad(integrand_f_phi, -1, 1)
            
            # Вычисляем норму полинома Лежандра (φ_k, φ_k)
            def integrand_phi_phi(t):
                return Pk(t)**2 * self.weight_t(t)
            
            integral_phi_phi, _ = integrate.quad(integrand_phi_phi, -1, 1)
            
            # Коэффициент по формуле: c_k = (f, φ_k) / (φ_k, φ_k)
            self.coefficients[k] = integral_f_phi / integral_phi_phi
        
        # Создаем функцию полинома
        def polynomial(x):
            t = self.x_to_t(x)
            result = np.zeros_like(x)
            for k in range(self.n + 1):
                Pk = legendre(k)
                result += self.coefficients[k] * Pk(t)
            return result
        
        self.polynomial = polynomial
        self._compute_errors()

    def _compute_errors(self):
        def fp_squared(x):
            return (self.f(x) - self.polynomial(x))**2 * self.weight_func(x)
        
        norm_fp_sq, _ = integrate.quad(fp_squared, self.a, self.b)
        
        self.absolute_error = np.sqrt(norm_fp_sq)
    
    def get_errors(self):
        return self.absolute_error
    
    def print_results(self):
        print("="*60)
        print("РЕЗУЛЬТАТЫ АППРОКСИМАЦИИ")
        print("="*60)
        print(f"Функция: f(x) на отрезке [{self.a}, {self.b}]")
        print(f"Степень полинома: {self.n}")
        print("\nКоэффициенты разложения:")
        print("c_k = (f, φ_k) / (φ_k, φ_k)")
        for i, coeff in enumerate(self.coefficients):
            print(f"  c_{i} = {coeff:.8f}")
        
        abs_error = self.get_errors()
        print(f"\nПогрешности:")
        print(f"  Абсолютная погрешность: E_{self.n}(f) = {abs_error:.8f}")
        
        print("="*60)

    def get_approximation_formula(self):
        terms = []
        for k in range(self.n + 1):
            if abs(self.coefficients[k]) > 1e-10:  # Игнорируем очень малые коэффициенты
                terms.append(f"{self.coefficients[k]:.6f}·P_{k}(t)")
        
        formula = "p(x) = " + " + ".join(terms)
        formula += f", где t = (2x - {self.a + self.b})/{self.b - self.a}"
        return formula

In [25]:
def f(x):
    return np.sqrt(x**2) + np.log(x**2)

a, b = 1, 6
n = 5

approx2 = LegendreApproximation(f, a=a, b=b, n=n, weight_func_vers=1)
approx2.print_results()
print(f"\nФормула: {approx2.get_approximation_formula()}")

РЕЗУЛЬТАТЫ АППРОКСИМАЦИИ
Функция: f(x) на отрезке [1, 6]
Степень полинома: 5

Коэффициенты разложения:
c_k = (f, φ_k) / (φ_k, φ_k)
  c_0 = 5.80022273
  c_1 = 2.05993318
  c_2 = -0.05579436
  c_3 = 0.00309682
  c_4 = -0.00013858
  c_5 = 0.00000515

Погрешности:
  Абсолютная погрешность: E_5(f) = 2.69103060

Формула: p(x) = 5.800223·P_0(t) + 2.059933·P_1(t) + -0.055794·P_2(t) + 0.003097·P_3(t) + -0.000139·P_4(t) + 0.000005·P_5(t), где t = (2x - 7)/5


**Задание 2.**

In [15]:
def f(x):
    return np.sqrt(x**2) + np.log(x**2)
    
a = -1
b = 0

approx3 = LegendreApproximation(f, a=a, b=b, n=10, weight_func_vers=2)
approx3.print_results()
print(f"\nФормула: {approx3.get_approximation_formula()}")

РЕЗУЛЬТАТЫ АППРОКСИМАЦИИ
Функция: f(x) на отрезке [-1, 0]
Степень полинома: 10

Коэффициенты разложения:
c_k = (f, φ_k) / (φ_k, φ_k)
  c_0 = -0.74967459
  c_1 = -2.13502901
  c_2 = 0.46204464
  c_3 = 0.38641149
  c_4 = 0.27210130
  c_5 = 0.19701499
  c_6 = 0.14908121
  c_7 = 0.11709631
  c_8 = 0.09470161
  c_9 = 0.07837204
  c_10 = 0.06606732

Погрешности:
  Абсолютная погрешность: E_10(f) = 2.15799406

Формула: p(x) = -0.749675·P_0(t) + -2.135029·P_1(t) + 0.462045·P_2(t) + 0.386411·P_3(t) + 0.272101·P_4(t) + 0.197015·P_5(t) + 0.149081·P_6(t) + 0.117096·P_7(t) + 0.094702·P_8(t) + 0.078372·P_9(t) + 0.066067·P_10(t), где t = (2x - -1)/1


In [16]:
a = 0
b = 1

approx3 = LegendreApproximation(f, a=a, b=b, n=10, weight_func_vers=2)
approx3.print_results()
print(f"\nФормула: {approx3.get_approximation_formula()}")

РЕЗУЛЬТАТЫ АППРОКСИМАЦИИ
Функция: f(x) на отрезке [0, 1]
Степень полинома: 10

Коэффициенты разложения:
c_k = (f, φ_k) / (φ_k, φ_k)
  c_0 = -0.74967459
  c_1 = 2.13502901
  c_2 = 0.46204464
  c_3 = -0.38641149
  c_4 = 0.27210130
  c_5 = -0.19701499
  c_6 = 0.14908121
  c_7 = -0.11709631
  c_8 = 0.09470161
  c_9 = -0.07837204
  c_10 = 0.06606732

Погрешности:
  Абсолютная погрешность: E_10(f) = 2.15799406

Формула: p(x) = -0.749675·P_0(t) + 2.135029·P_1(t) + 0.462045·P_2(t) + -0.386411·P_3(t) + 0.272101·P_4(t) + -0.197015·P_5(t) + 0.149081·P_6(t) + -0.117096·P_7(t) + 0.094702·P_8(t) + -0.078372·P_9(t) + 0.066067·P_10(t), где t = (2x - 1)/1
